# Corportate Bond Spreads Exploratory Notebook

## Importing dependencies

In [21]:
import pandas as pd
from datetime import datetime

In [31]:
# File path and loading data

# File paths
AAA = 'Corp_Bond_AAA.csv'
BBB = 'Corp_Bond_BBB.csv'
B = 'Corp_Bond_B.csv'

# Bond Data 
Corp_Bond_AAA_df= pd.read_csv(AAA)
Corp_Bond_BBB_df= pd.read_csv(BBB)
Corp_Bond_B_df= pd.read_csv(B)

combined_AAA_BBB = pd.merge(Corp_Bond_AAA_df,Corp_Bond_BBB_df, on='DATE')
combined_AAA_BBB

# Merging the Corporate B bond data
combined_AAA_BBB_B = pd.merge(combined_AAA_BBB,Corp_Bond_B_df, on='DATE')
combined_AAA_BBB_B

,DATE,BAMLC0A1CAAAEY,BAMLC0A4CBBBEY,BAMLH0A2HYBEY
0,3/19/2020,3.10,5.23,11.51
1,3/18/2020,2.96,4.82,10.86
2,3/17/2020,2.58,4.27,9.98
3,3/16/2020,2.28,3.86,9.76
4,3/13/2020,2.39,3.77,8.72
...,...,...,...,...
6054,1/7/1997,6.82,7.39,10.07
6055,1/6/1997,6.80,7.37,10.05
6056,1/3/1997,6.80,7.34,10.01
6057,1/2/1997,6.80,7.36,10.00


In [32]:
# Renaming the columns
combined_AAA_BBB_B.columns = ['Date','AAA Rating','BBB Rating', 'B Rating'] 

In [35]:
# Sorting by date 
combined_AAA_BBB_B = combined_AAA_BBB_B.sort_index(axis=0, ascending=False)
combined_AAA_BBB_B

# Converting the dates 
combined_AAA_BBB_B['Date'] = pd.to_datetime(combined_AAA_BBB_B['Date'])
combined_AAA_BBB_B

,Date,AAA Rating,BBB Rating,B Rating
6058,1996-12-31,6.72,7.26,10.01
6057,1997-01-02,6.80,7.36,10.00
6056,1997-01-03,6.80,7.34,10.01
6055,1997-01-06,6.80,7.37,10.05
6054,1997-01-07,6.82,7.39,10.07
...,...,...,...,...
4,2020-03-13,2.39,3.77,8.72
3,2020-03-16,2.28,3.86,9.76
2,2020-03-17,2.58,4.27,9.98
1,2020-03-18,2.96,4.82,10.86


In [44]:
# Bridging Treasury Data
treasury_yields = 'US_Yield_Curve.csv'

treasury_yield_df =pd.read_csv(treasury_yields)
# Ordering the dates

treasury_yield_df = treasury_yield_df.sort_index(axis=0, ascending=False)

#Getting the dates
treasury_yield_df['Date'] = pd.to_datetime(treasury_yield_df['Date'])
treasury_yield_df

# Keeping the same date range as above
dated_treasury_yield_df = treasury_yield_df.loc[(treasury_yield_df['Date']>="1997-01-02")]

dated_treasury_yield_df


#print(new_treasury_yield_df.isnull().sum())

,Date,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
5807,1997-01-02,NaN,NaN,5.19,5.35,5.63,5.97,6.13,6.30,6.45,6.54,6.85,6.75
5806,1997-01-03,NaN,NaN,5.17,5.34,5.60,5.95,6.11,6.28,6.42,6.52,6.84,6.74
5805,1997-01-06,NaN,NaN,5.17,5.31,5.61,5.97,6.12,6.30,6.44,6.54,6.86,6.77
5804,1997-01-07,NaN,NaN,5.16,5.30,5.61,5.98,6.14,6.32,6.47,6.57,6.89,6.80
5803,1997-01-08,NaN,NaN,5.16,5.30,5.61,6.01,6.17,6.34,6.49,6.60,6.92,6.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2020-03-16,0.25,0.25,0.24,0.29,0.29,0.36,0.43,0.49,0.67,0.73,1.10,1.34
3,2020-03-17,0.12,0.18,0.19,0.24,0.30,0.47,0.54,0.66,0.91,1.02,1.45,1.63
2,2020-03-18,0.04,0.03,0.02,0.08,0.21,0.54,0.66,0.79,1.08,1.18,1.60,1.77
1,2020-03-19,0.04,0.04,0.04,0.06,0.20,0.44,0.53,0.66,1.00,1.12,1.56,1.78


In [46]:
mid_curve_tsy_df = pd.DataFrame({'Date': dated_treasury_yield_df['Date'],
                                 '2 Year':dated_treasury_yield_df['2 YR'],
                                 '5 Year': dated_treasury_yield_df['5 YR'],
                                 '10 Year': dated_treasury_yield_df['10 YR'],
                                 '20 Year': dated_treasury_yield_df['20 YR']
                                })
mid_curve_tsy_df

,Date,2 Year,5 Year,10 Year,20 Year
5807,1997-01-02,5.97,6.30,6.54,6.85
5806,1997-01-03,5.95,6.28,6.52,6.84
5805,1997-01-06,5.97,6.30,6.54,6.86
5804,1997-01-07,5.98,6.32,6.57,6.89
5803,1997-01-08,6.01,6.34,6.60,6.92
...,...,...,...,...,...
4,2020-03-16,0.36,0.49,0.73,1.10
3,2020-03-17,0.47,0.66,1.02,1.45
2,2020-03-18,0.54,0.79,1.18,1.60
1,2020-03-19,0.44,0.66,1.12,1.56


In [55]:
corp_tsy_df = pd.merge_asof(combined_AAA_BBB_B,mid_curve_tsy_df,on='Date')
corp_tsy_df

,Date,AAA Rating,BBB Rating,B Rating,2 Year,5 Year,10 Year,20 Year
0,1996-12-31,6.72,7.26,10.01,NaN,NaN,NaN,NaN
1,1997-01-02,6.80,7.36,10.00,5.97,6.30,6.54,6.85
2,1997-01-03,6.80,7.34,10.01,5.95,6.28,6.52,6.84
3,1997-01-06,6.80,7.37,10.05,5.97,6.30,6.54,6.86
4,1997-01-07,6.82,7.39,10.07,5.98,6.32,6.57,6.89
...,...,...,...,...,...,...,...,...
6054,2020-03-13,2.39,3.77,8.72,0.49,0.70,0.94,1.31
6055,2020-03-16,2.28,3.86,9.76,0.36,0.49,0.73,1.10
6056,2020-03-17,2.58,4.27,9.98,0.47,0.66,1.02,1.45
6057,2020-03-18,2.96,4.82,10.86,0.54,0.79,1.18,1.60


In [ ]:
# Plotting the raw data 






